Previous
| Var      | Coeff |
|----------|-------|
|Hgt       |18.43|
|Spd       |17.48|
|dIQ       |11.92|
|oIQ       |9.27|
|Str       |8.05|
|Jmp       |7.17|
|3Pt       |6.58|
|Pss       |6.38|
|Drb       |5.94|
|End       |3.40|
|Dnk       |3.35|
|Reb       |0.88|
|FT.1      |0.60|
|Ins       |0.55|
|2Pt       |0.00|


Beta
| Var      | Coeff |
|----------|-------|
|Hgt       |19.88|
|Spd       |18.53|
|Jmp       |9.44|
|dIQ       |9.32|
|Str       |7.91|
|Drb       |7.63|
|oIQ       |6.64|
|3Pt       |6.61|
|Pss       |5.86|
|Ins       |2.22|
|Reb       |2.19|
|Dnk       |2.05|
|End       |1.70|
|FT.1      |0.00|
|2Pt       |0.00|



In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
df = pd.read_csv('beta_stats.csv')#big_stat
df = df[(df.G*df.MP > 82)]
time_weight = (1/np.array(df['MP']))
#df['DRE'] = (df['PTS'] + .2*df['TRB'] + 1.7*df['STL'] + .535*df['Blk'] + .5*df['AST'] - .9*df['FGA'] -.35*df['FTA'] - 1.4*df['TOV'])*time_weight*36
df['DRE'] = (-0.552*df['3PA']-0.719*(df['FGA']-df['3PA'])+0.79231*df['PTS'] + 0.1348*df['ORB'] +0.4*df['DRB'] + 1.68*df['STL'] + 0.7639*df['Blk'] + 0.544*df['AST'] - .16*df['FTA'] - 1.36*df['TOV'] -0.1084*df['PF'])*time_weight*48-8.4236
df.shape
_ = plt.hist(df.DRE,100)
plt.title('{}'.format(df.DRE.mean()))

In [ ]:
V = df[['Name','Ovr']].groupby('Name').max()

In [ ]:
#df = pd.read_csv('avg_stats.csv')
#groups = df.groupby(['pid','Season']).mean()

In [ ]:
list(df.columns).index('WS')

In [ ]:
(V >= 90).sum()

In [ ]:
vals = [76.9,63.9,56.2,50.8,46.6,43.2,40.3,37.8,35.6,33.6,31.8,30.2,28.7,27.3,26,24.8,23.6,22.6,21.5,20.6,19.7,18.8,18,17.2,16.4,15.6,14.9,14.3,13.6,13]
#df = df[df.Season >= 2030]

In [ ]:
vals2 = []
for i in range(30):
    agg = df[df['DraftPick'] == i+1].groupby('Name').sum()
    agg = agg.mean()
    #print(i,agg['WS'],agg['EWA'])
    vals2.append(agg['WS'])
#_ = plt.hist(['EWA'],20)
plt.plot(np.arange(30)+1,vals,label='NBA')
plt.plot(np.arange(30)+1,vals2,label='BBGM')
plt.xlabel('draft pick')
plt.xlim(1,30)
plt.ylabel('Mean Career Win Shares')
plt.legend()
plt.gca().set_facecolor('w')
#plt.gca().set_facecolor('w')
plt.savefig('draft.png',transparent=False)

In [ ]:
vals3 = []
valid_ages = set(list(range(19,36)))
for i in range(19,36):
    agg = df[df['Age'] == i]#.groupby('Name').sum()
    agg = agg.mean()
    #print(i,agg['WS'],agg['EWA'])
    vals3.append(agg['WS/48'])

In [ ]:
WS_REAL = defaultdict(list)
for YR in range(1980,2020):
    table = np.load('tables_{}.pkl'.format(YR))
    for team in table:
        stats = table[team]['advanced'][['Age','WS/48']]
        for item in stats.itertuples():
            WS_REAL[int(item[1])].append(item[2])

In [ ]:
vals4 = np.zeros(len(valid_ages))
for k,v in WS_REAL.items():
    if k in valid_ages:
        try:
            vals4[k-19] = np.array( [ _ for _ in v if type(_) == float ]).mean()
        except:
            print(np.array(v))

In [ ]:
plt.plot(range(19,36),vals4,label='NBA')
plt.plot(range(19,36),vals3,label='BBGM')
plt.xlabel('age')
plt.xlim(19,36)
plt.ylabel('Mean WS/48')
plt.legend()
plt.gca().set_facecolor('w')
#plt.gca().set_facecolor('w')
plt.savefig('ws48.png',transparent=False)

In [ ]:
dkeys = ['Hgt', 'Str', 'Spd', 'Jmp', 'End', 'Ins', 'Dnk', 'FT', '2Pt', '3Pt',
       'oIQ', 'dIQ', 'Drb', 'Pss', 'Reb']
new_ovr = []
nc = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])

new_ovr = np.array(df.iloc[:,-16:-1]) @ (nc/nc.sum())
#for row in df.iterrows():
#    new_ovr.append(sum([row[1][k]*v for k,v in zip(dkeys,nc)])/nc.sum())
#    break

plt.hist(new_ovr,100)
print(np.array(new_ovr).mean())
df.columns[-16:-1]

In [ ]:
plt.hist(df[df['+/-'] < -12]['Ovr'],15,density=True,label='Below -10',alpha=0.9)
plt.hist(df[df['+/-'] > +12]['Ovr'],15,density=True,label='Above +10',alpha=0.9)
plt.legend()

In [ ]:
nf = lambda x: (x-x.mean())/x.std()
y = np.sqrt(df['PER'] * np.maximum(1e-3,df['WS/48']))#df['+/-'] * time_weight * 36#45
y =  nf(df['WS/48'])
y = np.nan_to_num(y)

#y = df['+/-'] * time_weight * 36#45
#y.iloc[:,0] += (df.iloc[:,45]-0.074737)*50 #45
#y /=2 

X = df.iloc[:,-16:-1] # -17:-16
#X = X[[_ for _ in X.columns if _ != '+/-' ]]
#X.columns,y.columns,y.shape

In [ ]:
_ = plt.hist(y,50)

In [ ]:
y=pd.DataFrame(y)

In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=1,interaction_only=True)
scalerX = preprocessing.RobustScaler()
scalery = preprocessing.StandardScaler()
prescale_X = scalerX.fit_transform(X)
prescale_y = scalery.fit_transform(y)
#prescale_X = fexp.fit_transform(prescale_X)

In [ ]:
trials = 1
ts = []
prescale_X = (X-round(X.mean(0)))/round(X.std(0))
for i in range(trials):
    #clf = neural_network.MLPRegressor((36,5,24,36),'tanh',solver='adam',max_iter=1000)
    #clf = neural_network.MLPRegressor((32,4),'identity',solver='lbfgs',alpha=5e2,tol=1e-9)
    #clf = multioutput.MultiOutputRegressor(linear_model.SGDRegressor(penalty='l2',alpha=5e2,eta0=1e-6,tol=1e-12,max_iter=50,verbose=True))
    clf = multioutput.MultiOutputRegressor(linear_model.ElasticNetCV(cv=25,positive=True,fit_intercept=False))
    #clf = ensemble.ExtraTreesRegressor(8,criterion='mae',max_depth=3,verbose=1)
    #clf = multioutput.MultiOutputRegressor(svm.SVR())
    W = df.MP*df.G
    W /= W.mean()
    clf.fit(prescale_X,prescale_y)
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    err = np.linalg.norm(yt-y)
    ts.append((err,clf))

In [ ]:
if False:
    import statsmodels.api as sm
    clf_sm = sm.RLM(np.array(y['+/-']),prescale_X.astype(np.float),weights=W)
    res = clf_sm.fit()
    yt = scalery.inverse_transform(res.predict(prescale_X))
    err = np.linalg.norm(yt-y['+/-'])
    print(err)
    res.summary()

In [ ]:
ts = sorted(ts)[::1] # why not the biggest error
print(ts[0][0])
clf = ts[0][1]

In [ ]:
round(df.iloc[:,-16:-1].mean()),round(df.iloc[:,-16:-1].std())

In [ ]:
col_names = X.columns
#col_names = fexp.get_feature_names(X.columns)

for i,c in enumerate(y.columns):
    coeffs = clf.estimators_[i].coef_ 
    v = np.argsort(abs(coeffs))[::-1]
    print(c)
    coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v]
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('{:.2f} * {} + '.format(v,n),end='')
    print('| Var      | Coeff |')
    print('|----------|-------|')
    for v,n in sorted(coeffs2,reverse=True):
        print('|{:10s}|{:.2f}|'.format(n,100*v/coeffs.sum()))
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('\t{:25s}\t{:.2f}'.format(n,v))


In [ ]:
nc_M = []
for n,w in zip(col_names,clf.estimators_[0].coef_):
    if n == '1':
        continue
    w /= (clf.estimators_[0].coef_.sum())*0.01
    w = int(round(w))
    print(w,end=', ')
    nc_M.append(w)
nc_M = np.array(nc_M)

In [ ]:
round(X.std(0))

In [ ]:
coeff = clf.estimators_[i].coef_ 
coeff = np.round(coeff,2)
coeff[np.where(coeff <= 0)] = 0.01

pred = (X-round(X.mean(0)))/round(X.std(0)) @ coeff
MM = new_ovr.mean()
SS = new_ovr.std()/pred.std()
print(MM,SS)

new_ovr = np.array(new_ovr)
plt.hist(new_ovr,100)
plt.hist(pred*SS+MM,100,alpha=0.5)
#plt.ylim(0,50)
new_ovr.mean(),new_ovr.std(),coeff

In [ ]:
GEN_YEAR = 2019
with open('rosters/auto_roster_{}.json'.format(GEN_YEAR),'r') as fp:
    base = json.load(fp)
players = base['players']

In [ ]:
dkeys = [_.lower() for _ in ['Hgt', 'Stre', 'Spd', 'Jmp', 'Endu', 'Ins', 'Dnk', 'FT', 'fg', 'tp',
       'oIQ', 'dIQ', 'Drb', 'Pss', 'Reb']]
nc = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])
xm = round(X.mean(0))
xs = round(X.std(0))

In [ ]:
player_df = []
for p in players:
    if not 'name' in p:
        p['name'] = p['firstName'] + ' ' + p['lastName']
    d = {'name':p['name']}
    d.update(p['ratings'][0])
    d['ovr_og'] = sum([d[k]*v for k,v in zip(dkeys,nc)])/nc.sum()
    d['ovr_mn'] = sum([(d[k]-m)*v/s for k,v,m,s in zip(dkeys,coeff,xm,xs)])*SS + MM
    #d['ovr_mnr'] = sum([d[k]*v for k,v in zip(dkeys,nc)])/np.maximum(0,nc3).sum()

    player_df.append(d)

In [ ]:
pdf = pd.DataFrame(player_df).set_index('name')
pdf['diff'] = pdf['ovr_mn'] - pdf['ovr_og']
#pdf['diffr'] = pdf['ovr_mnr'] - pdf['ovr_og']


In [ ]:
pdf.sort_values(['ovr_og'],0,False)


In [ ]:
pdf.sort_values(['ovr_mn'],0,False)


In [ ]:
pdf.sort_values(['diff'],0,False)
#pdf.loc['Draymond Green']

In [ ]:
_ = plt.hist(pdf['diff'],40)

In [ ]:
pdf['diff'].mean(),pdf['diff'].std()

In [ ]:
plt.figure(figsize=(6,6),facecolor='w',edgecolor='w')
plt.scatter(pdf['ovr_og'],pdf['ovr_mn'],s=11,alpha=0.7)
#plt.scatter(new_ovr,pred*(SS)+MM+0.5,s=3,alpha=0.1)


plt.xlim(30,75)
plt.ylim(30,75)
plt.xlabel('old rating')
plt.ylabel('new rating')
plt.title("Rating change on nicidob 18-19 NBA Roster data")
plt.plot([30,75],[30,75],c='k')
plt.grid(True)

In [ ]:
df.columns

In [ ]:
df['GmBPM2'] = (-0.55*df['FGA'] + 2.43*df['3P']+ 1.39*(df['FGA']-df['3P']) + 0.79*df['FT'] -0.25*df['FTA'] \
                + 0.17*df['ORB'] - 0.27*df['DRB'] + 0.43*df['AST'] + 1.34*df['STL'] + 0.64*df['Blk']-0.95*df['TOV']-0.34*df['PF'])*time_weight*48

In [ ]:
_ = plt.hist(df['GmBPM2'],100)

In [ ]:
clf_bpm = linear_model.ElasticNetCV(cv=10,fit_intercept=True)

In [ ]:
clf_bpm

In [ ]:
df['+/-_min'] = df['+/-'] * time_weight * 48

In [ ]:
Xbpm = df.iloc[:,10:58]
Xbpm['2PA'] = Xbpm['FGA']-Xbpm['3PA']
Xbpm['2P'] = Xbpm['FG']-Xbpm['3P']

Ybpm = df['+/-_min']
#Xbpm = Xbpm[[_ for _ in Xbpm.columns if _ not in ['DRtg','ORtg','FG','LowPostFGA','AtRimFGA','TOV%', 'FG%','EWA', 'MidRangeFGP', '3PAr', 'AtRimFGP', 'LowPostFGP', '3P%', 'FT%', 'ORB%', 'FTr','WS','OWS','DWS','WS/48','PER', 'USG%', 'TRB%', 'TRB', 'TOV', 'STL', 'PF', 'ORB', 'MidRangeFGA', 'MidRangeFG', 'MP', 'LowPostFG', 'FT', 'DRB', 'Blk', 'BA', 'AtRimFG', '3PA', '3P',]]]
#Xbpm = Xbpm[[_ for _ in Xbpm.columns if _ not in ['DRtg','ORtg','PER','DWS','OWS','WS','WS/48','EWA', 'MidRangeFGA', 'MidRangeFG', 'AtRimFG','AtRimFGA','LowPostFGA', 'MP', '3PAr', 'LowPostFG', 'LowPostFGP', 'AtRimFGP', 'FTr', 'MidRangeFGP','FT%','TRB']]]
#Xbpm = Xbpm[[_ for _ in Xbpm.columns if _ not in  ['TRB%','ORB%','DRB%','TS%', 'FG', 'BLK%', 'BA', 'FGA', 'FG%', 'Blk', '3P', '3P%', 'FTA', 'FT', 'PF']]]
Xbpm = Xbpm[[_ for _ in Xbpm.columns if _ not in  ['DRtg','ORtg','2PA','2P','PF','Blk','3PA','3P%','FT','TS%','TRB','USG%','STL%','AST%','TRB%','ORB%','DRB%','BLK%', 'FG%','LowPostFGA','AtRimFGA','TOV%', '3P','FTr','FG%','EWA', 'MidRangeFGP', '3PAr', 'AtRimFGP', 'LowPostFGP', 'AtRimFG','MP', 'FT%', 'ORB%','WS','OWS','DWS','WS/48','PER', 'USG%', 'TRB%','LowPostFG','MidRangeFG','MidRangeFGA']]]

In [ ]:
for col in ['FGA','FTA','AST','PTS','TRB','ORB','DRB','TOV','STL','PF','FT','Blk','BA','3PA','3P','2PA','2P']:
    if col in Xbpm:
        Xbpm[col] = 48*Xbpm[col] * time_weight

In [ ]:
clf_bpm.fit(Xbpm,Ybpm)

In [ ]:
clf_bpm.intercept_

In [ ]:
clf_bpm.score(Xbpm,Ybpm)

In [ ]:
Ybpm_pred = clf_bpm.predict(Xbpm)

In [ ]:
plt.scatter(Ybpm,Ybpm_pred,s=3,alpha=0.5)
plt.figure()
plt.scatter(df['Ovr'],Ybpm_pred,s=3,alpha=0.5)
plt.figure()
plt.scatter(df['Ovr'],Ybpm,s=3,alpha=0.5)

In [ ]:
col_names = Xbpm.columns
#col_names = fexp.get_feature_names(X.columns)

coeffs = clf_bpm.coef_ 
v = np.argsort(abs(coeffs))[::-1]
print(c)
coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v]
#for v,n in sorted(coeffs2,reverse=True):
#    print('{:.2f} * {} + '.format(v,n),end='')
print('| Var      | Coeff |')
print('|----------|-------|')
for v,n in sorted(coeffs2,reverse=True):
    if abs(v)< 0.2:
        print(" '{}',".format(n),end='')
    print('{}, {:.2f}'.format(n,v))
    #print('|{:10s}|{:.2f}|'.format(n,100*v/coeffs.sum()))

#for v,n in sorted(coeffs2,reverse=True):
#    print('\t{:25s}\t{:.2f}'.format(n,v))
print('{}, {:.2f}'.format('Intercept',clf_bpm.intercept_))
print(Xbpm.shape[0])

In [ ]:
df['Ybpm_pred'] = Ybpm_pred

In [ ]:
df[['Ovr','+/-','Ybpm_pred','PER','EWA','WS/48','+/-_min','DRE']].corr()

In [ ]:
df

In [ ]:
import statsmodels.api as sm


In [ ]:
spector_data = Xbpm
spector_data = sm.add_constant(spector_data, prepend=False)

# Fit and summarize OLS model
mod = sm.OLS(Ybpm,spector_data)

res = mod.fit()

res.summary()

In [ ]:
Ybm_pres2 = res.predict(spector_data)

In [ ]:
df['Ybm_pres2'] = Ybm_pres2

In [ ]:
df[['Ovr','+/-','Ybpm_pred','Ybm_pres2','PER','EWA','WS/48','+/-_min','DRE','GmBPM2']].corr()

In [ ]:
plt.scatter(df['Ovr'],Ybm_pres2,s=3,alpha=0.5)

In [ ]:
plt.scatter(df['Ovr'],df['DRE'],s=3,alpha=0.5)
plt.figure()
plt.scatter(df['Ovr'],df['GmBPM2'],s=3,alpha=0.5)
plt.figure()
plt.scatter(df['Ybpm_pred'],df['DRE'],s=3,alpha=0.5)
plt.figure()
plt.scatter(df['Ybm_pres2'],df['DRE'],s=3,alpha=0.5)

In [ ]:
plt.scatter(Ybpm_pred,Ybm_pres2,s=3,alpha=0.5)

In [ ]:
Xbpm.corr().style.background_gradient()

In [ ]:
groups = df.groupby(['pid','Season'],as_index=False).mean()

In [ ]:
df[(df.pid == 0) & (df.Age > 30)][['Ovr','G','MP','Age','+/-_min','Ybpm_pred','Ybm_pres2','EWA','PER','WS/48',"DRE",'GmBPM2']]

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
df[(df.pid == 11329) & (df.Age > 24)  & (df.Age < 90) ][['Ovr','Age','+/-_min','Ybpm_pred','Ybm_pres2','EWA','PER','WS/48','DRE','GmBPM2']]

In [ ]:
df3 = df.groupby(['pid']).std()
df3[(df3.Ovr > 10) & ( df3.Season > 5)]

In [ ]:
df2 = groups._get_numeric_data().diff()

df3 = abs(df2[df2.pid == 0])
df3[ ['Ovr','Age','+/-_min','Ybpm_pred','Ybm_pres2','EWA','PER','DRE','GmBPM2']].mean()


In [ ]:
df

In [ ]:
groups

In [ ]:
df.groupby('pid').std().mean()

In [ ]:
df.std()

In [ ]:
len(df.pid.unique())

In [ ]:
df[df.Ovr >=90].shape